In [37]:
import pandas as pd
import numpy as np
from datetime import timedelta
from datetime import datetime
import matplotlib.pyplot as plt

trajectories = {} ##加载的每个路线的平均运行时间数据
volume = {}

In [166]:
def LoadData():
    df = pd.read_csv('../dataSets/training/training_20min_avg_travel_time.csv')
    start_time  = datetime.strptime("2016-07-19 00:00:00",'%Y-%m-%d %H:%M:%S')

    for idx in range(df.shape[0]):
        line = df.iloc[idx]
        route = str(line['intersection_id']+str(line['tollgate_id']))
        stime = datetime.strptime(line['time_window'][1:20],'%Y-%m-%d %H:%M:%S')
        day = (stime - start_time).days
        tw_n = (stime.hour * 60 + stime.minute)/20

        if route not in trajectories.keys():
            trajectories[route] = np.zeros(72*91).reshape(91,72)
        trajectories[route][day][tw_n] = line['avg_travel_time']

In [47]:
def LoadTrainAndTest(route):
    ##切分后7天作为测试数据
    train_data = trajectories[route][:-7]
    test_data = trajectories[route][-7:]
    return train_data,test_data

In [40]:
def TrueFalseListCombine(TFlist1,TFlist2):
    return [l1 and l2 for l1,l2 in zip(TFlist1,TFlist2)]

In [41]:
def ExceptionOutlier(data_list):
    '''
    shape(m,n),n timewindows,m days
    根据所给数据data_list中的时间窗的平均值和标准差来去除异常
    '''
    Mean = pd.DataFrame([np.mean(i) for i in data_list])
    mean_low = Mean > Mean.quantile(0.1)
    mean_up = Mean < Mean.quantile(0.9)
    TF = TrueFalseListCombine(mean_low.values,mean_up.values)
    mean_index = Mean[TF].index.values
    
    Std = pd.DataFrame([np.std(i) for i in data_list])
    std_low = Std > Std.quantile(0.1)
    std_up = Std < Std.quantile(0.9)
    TF = TrueFalseListCombine(std_low.values,std_up.values)
    std_index = Std[TF].index.values
    
    valid_index = list(set(mean_index)&set(std_index))
    
    return valid_index # i means minues i+1 day   

In [140]:
import pandas as pd
def method_mean(mean_time):
    ## 数据输出
    intersection = []
    tollgate = []
    time_window = []
    avg_time = []
    routes = ['A2','A3',"B1","B3","C1","C3"]
    for route in routes:
        n_day = 7
        start_time = datetime.strptime('2016-10-18 08:00:00',"%Y-%m-%d %H:%M:%S")
        for day in range(n_day):
            starttime = start_time + timedelta(days=day)
            i = 0
            for k in range(24,30): #(51,57)（24，30）
                time_window.append('\"[' + (starttime + timedelta(seconds=1200*i)).strftime("%Y-%m-%d %H:%M:%S")\
                                   + "," + (starttime + timedelta(seconds=1200*i+1200)).strftime("%Y-%m-%d %H:%M:%S") + ')\"')
                intersection.append(str(route[0]))
                tollgate.append(str(route[1]))
                avg_time.append(mean_time[route][k])
                i+=1

    for route in routes:
        n_day = 7
        start_time = datetime.strptime('2016-10-18 17:00:00',"%Y-%m-%d %H:%M:%S")
        for day in range(n_day):
            starttime = start_time + timedelta(days=day)
            i = 0
            for k in range(51,57): #(51,57)（24，30）
                time_window.append('\"[' + (starttime + timedelta(seconds=1200*i)).strftime("%Y-%m-%d %H:%M:%S")\
                                   + "," + (starttime + timedelta(seconds=1200*i+1200)).strftime("%Y-%m-%d %H:%M:%S") + ')\"')
                intersection.append(str(route[0]))
                tollgate.append(str(route[1]))
                avg_time.append(mean_time[route][k])
                i+=1            

    d = {"intersection_id":intersection,"tollgate_id":tollgate,"time_window":time_window,"avg_travel_time":avg_time}
    pdf = pd.DataFrame(data=d)
    pdf.to_csv('out.csv',index=False,columns=["intersection_id","tollgate_id","time_window","avg_travel_time"])
    return np.array(avg_time)

In [191]:
def predict_meantime(route):
    ##使用均值法
    ##预测10.18 到 10.24的8~10点以及17~18点。对两个时间段分别取训练数据
    ##对于8～10点的预测和其后的时间段是没有关系的
    #train_data,test_data = LoadTrainAndTest(route)
    train_data = trajectories[route]
    
    data_morning = train_data[:,18:24]
    idx_without_outlier = ExceptionOutlier(data_morning)
    data_morning_without_outlier = train_data[idx_without_outlier]
    mean_morning = np.mean(np.array(data_morning_without_outlier),axis=0)
    
    data_afternoon = train_data[:,45:51]
    idx_without_outlier = ExceptionOutlier(data_afternoon)
    data_afternoon_without_outlier = train_data[idx_without_outlier]
    mean_afternoon = np.mean(np.array(data_afternoon_without_outlier),axis=0)
    
    mean_data = np.zeros(72)
    mean_data[24:30] = mean_morning[24:30]
    mean_data[51:57] = mean_afternoon[51:57]
    
    return mean_data

In [189]:
def mape_t(true_data,predict_data):
    '''
    predict_data包含72个时间窗,对于每个route，其预测值为numpy.array格式
    predict_data为每一天预测的时间窗的预测平均运行时间,np.array类型
    12个时间窗
    '''  
    diff = true_data-predict_data
    summ = 0
    count = 0
    for x,d in zip(true_data,diff):
        if x == 0:
            continue
        else:
            count+=1
            summ += (abs(d)/x)
    
    #print "MAPE_Trajectories is %f" % np.mean(np.mean(np.abs(diff)/true_data,axis=1))
    print "MAPE_Trajectories is %f" % (summ/count)

In [192]:
def LoadTestData():
    test_data = []
    for route in trajectories.keys():
        test_route = trajectories[route][-7:][:,[24,25,26,27,28,29,51,52,53,54,55,56]].flatten()
        test_data.append(test_route)
    test_data = np.array(test_data).flatten()
    return test_data

In [193]:
LoadData()

In [194]:
out_data = {}
for route in trajectories.keys():
    out_data[route] = predict_meantime(route)

In [195]:
predict_data = method_mean(out_data)
test_data = LoadTestData()

In [179]:
trajectories['A2']

array([[   0.  ,   58.05,    0.  , ...,   54.59,   84.37,   67.96],
       [   0.  ,   37.89,   83.66, ...,   49.75,   55.15,   56.67],
       [  39.21,    0.  ,    0.  , ...,   39.15,   64.95,  122.84],
       ..., 
       [  59.6 ,   46.68,   42.41, ...,    0.  ,    0.  ,   59.24],
       [  59.43,   45.7 ,   45.34, ...,    0.  ,   42.05,   52.26],
       [  59.61,   40.03,    0.  , ...,   50.9 ,   79.21,   42.27]])

In [190]:
mape_t(test_data,predict_data)

MAPE_Trajectories is 0.367042
